https://arxiv.org/abs/1711.05772 / https://arxiv.org/abs/1802.04877

https://github.com/natashamjaques/magenta/blob/affective-reward/magenta/models/affective_reward/latent_gan.py

## Labeling data

In [1]:
#from pdb import set_trace
#set_trace()

In [2]:
import numpy as np

labels = np.empty([0, 2], dtype=int)
gens_keep = np.empty([0, ], dtype=object)
zs_keep = np.empty([0, 1, 128], dtype=float)

In [3]:
from pathlib import Path
vae_dir = Path('../../pytorch-text-vae') # https://github.com/iconix/pytorch-text-vae

n_samples = 100
temp = 0.2

%run -i $vae_dir/generate.py $vae_dir/reviews_sentences_state.pt reviews_sentences_stored_info.pkl --cache-path=$vae_dir/tmp --num-sample=$n_samples --temp=$temp

Namespace(cache_path='../../pytorch-text-vae/tmp', max_length=50, num_sample=100, print_z=False, saved_vae='../../pytorch-text-vae/reviews_sentences_state.pt', seed=None, stored_info='reviews_sentences_stored_info.pkl', temp=0.2, use_cuda=True)
Fetching cached info at ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl
Cache ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl loaded, total load time 0.6670348644256592
Found saved model ../../pytorch-text-vae/reviews_sentences_state.pt
MAX_SAMPLE: False; TRUNCATED_SAMPLE: True
Setting new random seed
---
there is a one of synth synth and and will be from their previous work
---
artist who continues to come along with the vocals and the gearing trying to look at every moment
---
artist listening to it comes along with the rest of artist and lana bliss
---
he continues to look at the release of the mash artist and a nu s disco
---
artist has come into a new mix of UNK and the UNK and has mash on more lately
---
artist wi

---
artist has been return with the one of the record and are trying to look at the way my way
---
UNK UNK UNK UNK UNK UNK UNK UNK is one on the soundtrack for our youtube feat
---
the la based electro pop aka UNK could have featured on the cover of their electro pop
---
the first track from your album right here to get along with a big kind of late here
---
artist is teams up with the nu disco percussion cant get stuck in the late night


In [4]:
list(zip(range(len(gens)), gens))

[(0, 'there is a one of synth synth and and will be from their previous work'),
 (1,
  'artist who continues to come along with the vocals and the gearing trying to look at every moment'),
 (2,
  'artist listening to it comes along with the rest of artist and lana bliss'),
 (3,
  'he continues to look at the release of the mash artist and a nu s disco'),
 (4,
  'artist has come into a new mix of UNK and the UNK and has mash on more lately'),
 (5,
  'artist will be going will take on the below and i look at the link below'),
 (6,
  'as speaking of the track which of UNK and trying to come them as a version of UNK or guitar'),
 (7,
  'artist is for a new single called day know how we will going on the track below or feat'),
 (8,
  'in the sort an indie pop music though we we lost in making music even more of of more lyrics'),
 (9, 'soon as part of the past week when hes released on your side'),
 (10,
  'swedish producer UNK has managed to hit along with all also soundtrack to UNK your wh

In [5]:
#new_labels = np.array([(1, -1), (10, -1)])

# -1 is "bad" because it contains a banned word, 1 is "good"
banned = ['below']
labels = np.ones(n_samples, dtype=int)
gens_lose = set([i for b in banned for i in np.where([b in g.split() for g in gens])[0]])
gens_keep = list(set(range(len(gens))) - gens_lose)
labels[gens_keep] = -1
zs_keep = zs

labels

array([-1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [6]:
#labels = np.concatenate((labels, new_labels))
#gens_keep = np.concatenate((gens_keep, np.array(gens)[np.array([i for i, l in new_labels])]))
#zs_keep = np.concatenate((zs_keep, np.array([z.cpu().numpy() for z in zs])[np.array([i for i, l in new_labels])]))

#gens_keep

In [7]:
#gens_keep.shape, zs_keep.shape, labels.shape

In [8]:
#np.get_printoptions()

In [9]:
#np.set_printoptions(threshold=np.inf)
#zs_keep

In [10]:
# -1 is "bad", 1 is "good"
#labels = np.ones(n_samples)

# for now, make up some random labels
#np.random.seed(42)
#ones = np.random.rand(n_samples) < 0.2
#labels[ones] = -1

#labels = np.array([l for i, l in labels], dtype=float)
#labels

In [11]:
from collections import Counter
c1 = Counter([word for gen in gens for word in gen.split()])
[(b, c1[b]) for b in banned], c1.most_common(25)

([('below', 5)],
 [('the', 98),
  ('UNK', 86),
  ('of', 74),
  ('and', 62),
  ('to', 61),
  ('a', 60),
  ('artist', 36),
  ('in', 29),
  ('with', 26),
  ('on', 25),
  ('as', 25),
  ('is', 23),
  ('one', 20),
  ('be', 17),
  ('from', 17),
  ('trying', 17),
  ('has', 17),
  ('it', 16),
  ('we', 16),
  ('for', 15),
  ('that', 15),
  ('their', 14),
  ('track', 14),
  ('this', 14),
  ('will', 13)])

## Data

In [12]:
batch_size = 16

In [13]:
from fastai.dataset import *

class LatentDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return A(self.x[idx], self.y[idx])
    def __len__(self): return len(self.x)
    
ds = LatentDataset(zs_keep, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [14]:
md.trn_ds[0]

[array([[-1.41267, -0.31883,  0.5313 ,  0.58262, -0.50945,  1.47321, -1.20368, -0.41302,  0.02329,  0.30825,
          0.58677, -0.59381,  1.08877, -0.75519, -0.65374, -2.59229, -0.62915, -0.28631, -0.10457,  2.32935,
          0.47541,  1.22917, -0.30757,  1.09619, -0.70228, -0.16351,  0.46556,  0.17515,  0.56253,  1.08933,
         -2.38834,  0.63997,  1.13505,  0.90841,  2.31919,  1.02084, -1.11749, -0.47312,  0.30283,  0.63565,
         -0.45139, -0.58845,  0.21838,  0.73886,  0.9245 ,  0.85827,  0.35686, -0.27496,  0.31194,  0.96879,
          0.67177,  0.43427,  0.55612,  0.19467, -0.0795 ,  1.2753 ,  1.04369,  0.17302,  1.24513, -1.09378,
          0.42322, -1.26422,  0.84987, -0.67009,  0.27917,  1.32686, -0.69538, -0.8589 ,  0.90375, -0.75259,
         -0.51699,  1.57829, -0.50048,  0.69828, -1.87919, -1.06044,  1.27093,  0.16094,  0.14646,  0.6237 ,
         -0.90125, -2.10465, -0.94218,  0.34538, -1.82606,  3.50424,  0.30298,  0.15515, -0.55099,  0.63468,
          0.36419, 

## Model

In [15]:
n_latent = 128
n_hidden = 1024
lr = 3e-4

In [16]:
import torch.optim as optim
import torch.nn as nn

class LCGAN_D(nn.Module):
    '''Discriminator'''
    def __init__(self, n_latent, n_hidden=n_hidden, n_output=1):
        super(LCGAN_D, self).__init__()
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        self.h2o = nn.Linear(n_hidden, n_output)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        v = self.sigmoid(self.h2o(x))
        
        return v

class LCGAN_G(nn.Module):
    '''Generator'''
    def __init__(self, n_latent, n_hidden=n_hidden):
        super(LCGAN_G, self).__init__()
        self.n_latent = n_latent
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        # hidden-to-gating mechanism
        self.h2g = nn.Linear(n_hidden, 2*n_latent)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        x = self.h2g(x)
        
        # gating mechanism: allow network to remember/forget
        # what it wants to about the original z and x
        z_mid = x[:, self.n_latent:]
        gates = self.sigmoid(x[:, :self.n_latent])
        dz = gates * z_mid # TODO: why naming?
        z_prime = (1 - gates)*z + dz
        
        return z_prime

In [17]:
class LCGAN(nn.Module):
    def __init__(self, D, G, batch_size=batch_size):
        super(LCGAN, self).__init__()
        self.batch_size = batch_size
        
        self.D = D
        self.G = G

    def train(self):
        self.D.train()
        self.G.train()
        
    def forward(self, z=None):       
        if z is not None:
            # train discriminator
            v = self.D(z)
            return v
        else: # train GAN
            # gaussian random noise
            z_prior = torch.randn(self.batch_size, self.G.n_latent).cuda()
            
            z_prime = self.G(z_prior)
            v_prime = self.D(z_prime)
            
            return v_prime

## Training

In [18]:
gan = LCGAN(LCGAN_D(n_latent), LCGAN_G(n_latent)).cuda()

In [19]:
import fastai as fast

fast.core.set_trainable(gan.D, True)
fast.core.set_trainable(gan.G, True)

opt_d = optim.Adam(gan.D.parameters(), lr=lr)
opt_g = optim.Adam(gan.G.parameters(), lr=lr)

In [20]:
# test what the GAN is doing before any training

def generate(z):
    generated = vae.decoder.generate(z, 50, temp, True)
    generated_str = model.float_word_tensor_to_string(output_side, generated)

    EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

    if generated_str.endswith(EOS_str):
        generated_str = generated_str[:-5]

    # flip it back
    return generated_str[::-1]
    
generate(gan.G(torch.randn(1, n_latent).cuda()))

'looking for a listen for the UNK but they the gearing up for coming alone'

In [21]:
for i in range(10):
    print(generate(gan.G(torch.randn(1, n_latent).cuda())))

in the release of UNK and serves as a listen to his first feat
he back with a remix of UNK and the manages to UNK UNK times
along a serves as one of the UNK has trying to in his first feat
one of his debut production where you to to reminiscent of a brand new help because its no matter
after listening an artist a stuck in new song because its than feat
artist has been back have a UNK or later last UNK and the stuck in their headline show
artist to come out of a week and and them of new taken from last times
artist trying to stripped work in house where are gearing up with a few alone
it seems would be taken from the music and as part such a look at times
UNK producer UNK is and in the sort of trying to come along with an last times


In [22]:
#from tqdm import tqdm

# adapted from: https://github.com/fastai/fastai/blob/master/courses/dl2/wgan.ipynb
def train(n_iter, alternate=False, first=False):
    gen_iters = 0
    for epoch in trange(n_iter):
        gan.train()
        data_iter = iter(md.trn_dl)
        i, n = 0, len(md.trn_dl)
        
        def train_G():
            ''' Train generator '''
            nonlocal gen_iters
            
            fast.core.set_trainable(gan.D, False)
            fast.core.set_trainable(gan.G, True)

            gan.G.zero_grad()

            #print(i, n)
            loss_g = - torch.log(gan()).mean()
            loss_g.backward()
            opt_g.step()
            gen_iters += 1
            
            return loss_g

        def train_D():
            ''' Train discriminator '''
            nonlocal i
            
            fast.core.set_trainable(gan.D, True)
            fast.core.set_trainable(gan.G, False)
            d_iters = 100 if (first and (gen_iters < 25) or (gen_iters % 500 == 0)) else 3
            j = 0

            while (j < d_iters) and (i < n):
                j += 1; i += 1
                batch = next(data_iter)
                #print(j, i, batch[0].size(), batch[1].size())
                z_real = batch[0].cuda()
                v = gan(z_real)

                gan.D.zero_grad()

                loss_d = - (batch[1] * torch.log(v) + (1.0-batch[1]) * torch.log(1.0 - v)).mean()
                loss_d.backward()
                opt_d.step()
                
                pbar.update()
                
            return loss_d
        
        with tqdm(total=n) as pbar:
            while i < n:
                if alternate:
                    # train discriminator
                    loss_d = train_D()
                    # then train generator a little bit
                    loss_g = train_G()
                else:
                    # train generator only
                    i += 1
                    loss_g = train_G()
                    pbar.update()
        
        if alternate:
            print(f'Loss_D {to_np(loss_d)}; Loss_G {to_np(loss_g)}; ')
        else:
            print(f'Loss_G {to_np(loss_g)}; ')

In [23]:
def train_and_generate(gan, n_epoch, alternate=False, n_sample=10):
    train(n_epoch, alternate)
    for i in range(n_sample):
        print(generate(gan.G(torch.randn(1, n_latent).cuda())))

In [24]:
train_and_generate(gan, 1, alternate=True)

100%|██████████| 7/7 [00:00<00:00, 235.22it/s]
Loss_D -3.201712131500244; Loss_G 2.297457218170166; 
below and according to a new version of the UNK which is just under our ago
artist has to teams up with UNK and one of the track be taken from our show
speaking of the track taken from UNK and a track to be taken from now i times
artist has been listening to the UNK and dose of what will take under his times
artist is gearing up with the sort of songs their ready for many times
artist has a listening of the UNK and thats exactly what i again
UNK are continues to take on a brand new UNK and we just listen from your than feat
UNK continues to is such a bit of artist and stuck in the rather as feat
as part of here and listening to a brand new track that theres no feat
artist has continues to serves as a cover of even when it comes in my store


In [25]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 422.44it/s]
Loss_G 1.7765755653381348; 
recorded in the dose of managed to giving it into a few times
artist has to a serves as remix of the week and it would be taken from any feat
it comes to a fresh a stuck in the thank you grab your show
you with a remix of UNK and of trying to get stuck under your head
and serves to a remix of artist who has to the stuck in city
it feels gets a remix of UNK and a perfect trying to look at times
according to a a remix of artist who and take on the yourself last store
UNK is one of the knack for trying to get a white feat
manages to serves as part of the right now and lost in the new york city
UNK to a part of the single below and single from free feat


In [26]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 414.45it/s]
Loss_G 1.352441430091858; 
it seems dropped a remix of UNK and the and will take on your coming feat
artist back to a long as part of the track and blown taken from their at times
and serves and one of soulful track from trying to for a many feat
the UNK and a sort is trying to look at the few times
as part of of UNK and as a listen get an pretty times
according to to UNK with a remix of the that will take on your at times
artist continues to long as part of the track and blown away a previous times
the remix of UNK and and take on a new trying to or feat
UNK has taken from the UNK and a sort of trying to look from any times
as part of the UNK and will take on your at feat


In [27]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 419.45it/s]
Loss_G 1.0633442401885986; 
artist has a be blown away of UNK and the better known for many feat
as a UNK and trying to be one of the UNK and from no feat
it seems to be such as part of artist and the stuck on your four feat
as one of the UNK and it would and taken from your every feat
serves as one of the UNK as a listen to your many feat
artist is teams up with the remix of deep UNK and you to be departure from every feat
artist artist continues to long as part of the life and taken from their live feat
artist is a listen in the kind the track and taken from their portland every feat
artist continues a long as part of the week and and some departure from every feat
as one of the UNK and a the trying to look theres every feat


In [28]:
g2 = []

for i in range(100):
    g2.append(generate(gan.G(torch.randn(1, n_latent).cuda())))
    
c2 = Counter([word for gen in g2 for word in gen.split()])
[(b, c2[b]) for b in banned], c2.most_common(25)

([('below', 2)],
 [('the', 109),
  ('and', 96),
  ('feat', 96),
  ('of', 93),
  ('to', 90),
  ('UNK', 85),
  ('a', 58),
  ('one', 57),
  ('as', 57),
  ('from', 54),
  ('every', 44),
  ('artist', 36),
  ('trying', 31),
  ('taken', 31),
  ('track', 26),
  ('it', 24),
  ('is', 23),
  ('be', 23),
  ('serves', 21),
  ('your', 20),
  ('on', 18),
  ('with', 17),
  ('take', 17),
  ('would', 16),
  ('continues', 14)])

In [29]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 429.50it/s]
Loss_G 0.9722763299942017; 
on this UNK and as one of the trying to give it for many times
with a UNK and and as one of the trying to giving us from every times
with a UNK and the kind of trying would be taken from your times
he continues to serves as one of the UNK and giving us from every feat
serves as one of the UNK and a perfect trying to look many feat
listened with a new track called UNK and as one of the trying to look many times
taken from the UNK and a sort is trying to on your every times
i continues to to a UNK and one to the it for at times
artist has teams up with a remix of the UNK and it would be taken from many feat
with a UNK and serves as one of the trying to give it for many feat


In [30]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 425.47it/s]
Loss_G 0.9507181644439697; 
later called UNK and and one of its trying to a departure from every feat
artist continues to come along with a UNK and the sort new trying to look at feat
artist is back to a right now at the one from their portland no times
artist is back in the link below and and as one from their portland every feat
away from UNK and the one and a new trying to look real times
to UNK and serves as one of the track where you to better known for at feat
and UNK is one of of trying to come along with each your times
artist continues to serves as part of the track be taken on your every times
artist has featured from the sort of trying to give it into a many times
i seems to as one of the UNK and soulful track from every times


In [31]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 429.03it/s]
Loss_G 0.9378024935722351; 
such as one of the UNK and it would be taken from every feat
he back gets a remix of UNK be taken from the ready for at feat
artist is according to to serves as one of the UNK and it would be taken from many feat
artist has teams up with a one of the UNK and it would be taken from every feat
UNK and serves and a sort of trying to look to the stuck in feat
with a new remix of UNK artist has been trying will take from the portland every feat
artist feels with a UNK as one of the trying might being every feat
artist has teams up and a one of the track which was trying to keep your every feat
reminiscent of serves as of UNK and listen to the ready for many feat
as reminiscent of serves on new UNK a listen to the ready for many feat


In [32]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 435.45it/s]
Loss_G 0.926938533782959; 
as one of the UNK and it would be taken from their coming feat
according and the reminiscent of the track that will take keep your every feat
artist is a UNK and listen on the remix of what i look at your feat
artist continues to long as part of the track and blown taken from every feat
artist has teams up with a UNK and one of new trying would be taken from every feat
as part of the UNK trying to come along with a the trying to look at feat
UNK is taken from the sort of trying to give it into an real feat
artist is with a taken of the UNK and of trying would be taken from their at times
with a UNK and serves as one of the trying to give it keep or times
as one of the UNK and it will take keep at every times


In [33]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 430.37it/s]
Loss_G 0.9276492595672607; 
with a UNK and remix of the track it be taken from their portland coming feat
in the likes of UNK and listened to a new taken on your every feat
a new track called UNK and as one of the trying to look theres every feat
artist are along with an new remix of UNK and a listen of the UNK theres every feat
with an new remix of UNK and a according to the better ready for at feat
listen to serves as one of the UNK and it would be taken from every times
artist are along with a new track UNK and look of the was trying to keep your feat
back to a long a part of link artist and stuck in the along theres every feat
right UNK as one of of trying to give it into some every feat
serves as a UNK and the sort the trying to look theres every feat


In [34]:
g3 = []

for i in range(100):
    g3.append(generate(gan.G(torch.randn(1, n_latent).cuda())))
    
c3 = Counter([word for gen in g3 for word in gen.split()])
[(b, c3[b]) for b in banned], c3.most_common(25)

([('below', 0)],
 [('the', 104),
  ('UNK', 95),
  ('of', 92),
  ('and', 92),
  ('feat', 90),
  ('to', 88),
  ('a', 72),
  ('one', 61),
  ('every', 51),
  ('trying', 45),
  ('as', 42),
  ('from', 37),
  ('many', 37),
  ('it', 32),
  ('with', 30),
  ('artist', 29),
  ('track', 29),
  ('taken', 26),
  ('is', 24),
  ('be', 24),
  ('serves', 19),
  ('for', 19),
  ('look', 18),
  ('up', 17),
  ('has', 15)])